In [88]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
import numpy as np
from docplex.mp.model import  Model

In [79]:
channels =  pd.DataFrame(data=[("gift", 20.0, 0.20), ("newsletter", 15.0, 0.05), ("seminar", 23.0, 0.30)], columns=["channel", "cost", "factor"])
display(channels)


products = ["Savings", "Mortgage", "Pension"]
product_value = [200, 300, 400]
budget_share = [0.2, 0.5, 0.3]
products = pd.DataFrame([products,product_value,budget_share]).transpose()
products.columns = ['product','value','budget_share']
display(products)



names =pd.DataFrame.from_dict( {
    139987 : "Guadalupe J. Martinez", 140030 : "Michelle M. Lopez", 140089 : "Terry L. Ridgley", 
    140097 : "Miranda B. Roush", 139068 : "Sandra J. Wynkoop", 139154 : "Roland Guérette", 139158 : "Fabien Mailhot", 
    139169 : "Christian Austerlitz", 139220 : "Steffen Meister", 139261 : "Wolfgang Sanger",
    139416 : "Lee Tsou", 139422 : "Sanaa' Hikmah Hakimi", 139532 : "Miroslav Škaroupka", 
    139549 : "George Blomqvist", 139560 : "Will Henderson", 139577 : "Yuina Ohira", 139580 : "Vlad Alekseeva", 
    139636 : "Cassio Lombardo", 139647 : "Trinity Zelaya Miramontes", 139649 : "Eldar Muravyov", 139665 : "Shu T'an", 
    139667 : "Jameel Abdul-Ghani Gerges", 139696 : "Zeeb Longoria Marrero", 139752 : "Matheus Azevedo Melo", 
    139832 : "Earl B. Wood", 139859 : "Gabrielly Sousa Martins", 139881 : "Franca Palermo"}, orient = 'index').reset_index()

names.columns = ['customerid','name']
display(names.head(2))


offers = pd.DataFrame([(139987, "Pension", 0.13221, "Mortgage", 0.10675), (140030, "Savings", 0.95678, "Pension", 0.84446), (140089, "Savings", 0.95678, "Pension", 0.80233), 
                        (140097, "Pension", 0.13221, "Mortgage", 0.10675), (139068, "Pension", 0.80506, "Savings", 0.28391), (139154, "Pension", 0.13221, "Mortgage", 0.10675), 
                        (139158, "Pension", 0.13221, "Mortgage", 0.10675),(139169, "Pension", 0.13221, "Mortgage", 0.10675), (139220, "Pension", 0.13221, "Mortgage", 0.10675), 
                        (139261, "Pension", 0.13221, "Mortgage", 0.10675), (139416, "Pension", 0.13221, "Mortgage", 0.10675), (139422, "Pension", 0.13221, "Mortgage", 0.10675), 
                        (139532, "Savings", 0.95676, "Mortgage", 0.82269), (139549, "Savings", 0.16428, "Pension", 0.13221), (139560, "Savings", 0.95678, "Pension", 0.86779), 
                        (139577, "Pension", 0.13225, "Mortgage", 0.10675), (139580, "Pension", 0.13221, "Mortgage", 0.10675), (139636, "Pension", 0.13221, "Mortgage", 0.10675), 
                        (139647, "Savings", 0.28934, "Pension", 0.13221), (139649, "Pension", 0.13221, "Mortgage", 0.10675), (139665, "Savings", 0.95675, "Pension", 0.27248), 
                        (139667, "Pension", 0.13221, "Mortgage", 0.10675), (139696, "Savings", 0.16188, "Pension", 0.13221), (139752, "Pension", 0.13221, "Mortgage", 0.10675), 
                        (139832, "Savings", 0.95678, "Pension", 0.83426), (139859, "Savings", 0.95678, "Pension", 0.75925), (139881, "Pension", 0.13221, "Mortgage", 0.10675)],
                        columns=["customerid", "Product1", "Confidence1", "Product2", "Confidence2"])

display( offers.head(2)   )  


available_budget = 500

,channel,cost,factor
0,gift,20.0,0.20
1,newsletter,15.0,0.05
2,seminar,23.0,0.30


,product,value,budget_share
0,Savings,200,0.2
1,Mortgage,300,0.5
2,Pension,400,0.3


,customerid,name
0,139987,Guadalupe J. Martinez
1,140030,Michelle M. Lopez


,customerid,Product1,Confidence1,Product2,Confidence2
0,139987,Pension,0.13221,Mortgage,0.10675
1,140030,Savings,0.95678,Pension,0.84446


In [70]:
mdl = Model(name="marketing_campaign")

#### Define the decision variables
- The integer decision variables `channel_vars`, represent whether or not a customer will be made an offer for a particular product via a particular channel.
- The integer decision variable `total_offers` represents the total number of offers made.
- The continuous variable `budget_spent` represents the total cost of the offers made.

In [82]:
def continuous_var_series(df, mdl,**kargs):
    return pd.Series(mdl.continuous_var_list(df.index, **kargs), index = df.index)

class CplexSum():
    """Function class that adds a series of dvars into a cplex sum expression.
    To be used as a custom aggregation in a groupby.
    Usage:
        df2 = df1.groupby(['a']).agg({'xDVar':CplexSum(engine.mdl)}).rename(columns={'xDVar':'expr'})

    Sums the dvars in the 'xDVar' column into an expression
    """
    def __init__(self, mdl):
        self.mdl = mdl
    def __call__(self, dvar_series):
        return self.mdl.sum(dvar_series)

In [74]:
customer_offers = pd.merge (   pd.merge(products['product'],channels['channel'] , how = 'cross') , offers['customerid'], how = 'cross').set_index(['product','channel','customerid'], verify_integrity = True) 

customer_offers['channel_vars']= continuous_var_series(customer_offers,mdl, name = 'X_offer')

customer_offers = customer_offers.reset_index(drop = False)

customer_offers.head()

,product,channel,customerid,channel_vars
0,Savings,gift,139987,X_offer_Savings_gift_139987
1,Savings,gift,140030,X_offer_Savings_gift_140030
2,Savings,gift,140089,X_offer_Savings_gift_140089
3,Savings,gift,140097,X_offer_Savings_gift_140097
4,Savings,gift,139068,X_offer_Savings_gift_139068


In [80]:
products = products.set_index(['product'], verify_integrity = True)
products['budget_per_product']= continuous_var_series(products,mdl, name = 'X_budget')
products = products.reset_index(drop = False)

products

,product,value,budget_share,budget_per_product
0,Savings,200,0.2,X_budget_Savings
1,Mortgage,300,0.5,X_budget_Mortgage
2,Pension,400,0.3,X_budget_Pension


In [81]:
total_offers = mdl.integer_var(name="total_offers")
budget_spent = mdl.continuous_var(name="spent")

## Set up the constraints
- Offer only one product per customer.
- Compute the budget and set a maximum on it.
- Compute the number of offers to be made.

### Only 1 product is offered to each customer     


In [91]:
offer_per_customer = customer_offers[['customerid','channel_vars']].groupby('customerid').agg(CplexSum(mdl))

offer_per_customer.head(1)

for row in offer_per_customer.itertuples():
    mdl.add_constraint(row.channel_vars <= 1)

### total offers is simply sum of channel vars

In [93]:
mdl.add( total_offers == mdl.sum(customer_offers['channel_vars']) );

### per product budgets is equal to sum of products' budgets in different channels and persons

In [105]:
df = pd.merge(customer_offers, channels, on = ['channel'] )

df['offering_budget'] = df['channel_vars'] * df['cost']

df.head(1)

product_budget = df[['product','offering_budget']].groupby(['product']) .agg(CplexSum(mdl)).reset_index()

df1 = pd.merge(product_budget, products, on = ['product'])

display(df1)

for row in df1.itertuples():
    mdl.add_constraint(row.offering_budget == row.budget_per_product)

,product,offering_budget,value,budget_share,budget_per_product
0,Mortgage,20X_offer_Mortgage_gift_139987+20X_offer_Mortgage_gift_140030+20X_offer_Mortgage_gift_140089+20X_offer_Mortgage_gift_140097+20X_offer_Mortgage_gift_139068+20X_offer_Mortgage_gift_139154+20X_offer_Mortgage_gift_139158+20X_offer_Mortgage_gift_139169+20X_offer_Mortgage_gift_139220+20X_offer_Mortgage_gift_139261+20X_offer_Mortgage_gift_139416+20X_offer_Mortgage_gift_139422+20X_offer_Mortgage_gift_139532+20X_offer_Mortgage_gift_139549+20X_offer_Mortgage_gift_139560+20X_offer_Mortgage_gift_139577+20X_offer_Mortgage_gift_139580+20X_offer_Mortgage_gift_139636+20X_offer_Mortgage_gift_139647+20X_offer_Mortgage_gift_139649+20X_offer_Mortgage_gift_139665+20X_offer_Mortgage_gift_139667+20X_offer_Mortgage_gift_139696+20X_offer_Mortgage_gift_139752+20X_offer_Mortgage_gift_139832+20X_offer_Mortgage_gift_139859+20X_offer_Mortgage_gift_139881+15X_offer_Mortgage_newsletter_139987+15X_offer_Mortgage_newsletter_140030+15X_offer_Mortgage_newsletter_140089+15X_offer_Mortgage_newsletter_140097+15X_offer_Mortgage_newsletter_139068+15X_offer_Mortgage_newsletter_139154+15X_offer_Mortgage_newsletter_139158+15X_offer_Mortgage_newsletter_139169+15X_offer_Mortgage_newsletter_139220+15X_offer_Mortgage_newsletter_139261+15X_offer_Mortgage_newsletter_139416+15X_offer_Mortgage_newsletter_139422+15X_offer_Mortgage_newsletter_139532+15X_offer_Mortgage_newsletter_139549+15X_offer_Mortgage_newsletter_139560+15X_offer_Mortgage_newsletter_139577+15X_offer_Mortgage_newsletter_139580+15X_offer_Mortgage_newsletter_139636+15X_offer_Mortgage_newsletter_139647+15X_offer_Mortgage_newsletter_139649+15X_offer_Mortgage_newsletter_139665+15X_offer_Mortgage_newsletter_139667+15X_offer_Mortgage_newsletter_139696+15X_offer_Mortgage_newsletter_139752+15X_offer_Mortgage_newsletter_139832+15X_offer_Mortgage_newsletter_139859+15X_offer_Mortgage_newsletter_139881+23X_offer_Mortgage_seminar_139987+23X_offer_Mortgage_seminar_140030+23X_offer_Mortgage_seminar_140089+23X_offer_Mortgage_seminar_140097+23X_offer_Mortgage_seminar_139068+23X_offer_Mortgage_seminar_139154+23X_offer_Mortgage_seminar_139158+23X_offer_Mortgage_seminar_139169+23X_offer_Mortgage_seminar_139220+23X_offer_Mortgage_seminar_139261+23X_offer_Mortgage_seminar_139416+23X_offer_Mortgage_seminar_139422+23X_offer_Mortgage_seminar_139532+23X_offer_Mortgage_seminar_139549+23X_offer_Mortgage_seminar_139560+23X_offer_Mortgage_seminar_139577+23X_offer_Mortgage_seminar_139580+23X_offer_Mortgage_seminar_139636+23X_offer_Mortgage_seminar_139647+23X_offer_Mortgage_seminar_139649+23X_offer_Mortgage_seminar_139665+23X_offer_Mortgage_seminar_139667+23X_offer_Mortgage_seminar_139696+23X_offer_Mortgage_seminar_139752+23X_offer_Mortgage_seminar_139832+23X_offer_Mortgage_seminar_139859+23X_offer_Mortgage_seminar_139881,300,0.5,X_budget_Mortgage
1,Pension,20X_offer_Pension_gift_139987+20X_offer_Pension_gift_140030+20X_offer_Pension_gift_140089+20X_offer_Pension_gift_140097+20X_offer_Pension_gift_139068+20X_offer_Pension_gift_139154+20X_offer_Pension_gift_139158+20X_offer_Pension_gift_139169+20X_offer_Pension_gift_139220+20X_offer_Pension_gift_139261+20X_offer_Pension_gift_139416+20X_offer_Pension_gift_139422+20X_offer_Pension_gift_139532+20X_offer_Pension_gift_139549+20X_offer_Pension_gift_139560+20X_offer_Pension_gift_139577+20X_offer_Pension_gift_139580+20X_offer_Pension_gift_139636+20X_offer_Pension_gift_139647+20X_offer_Pension_gift_139649+20X_offer_Pension_gift_139665+20X_offer_Pension_gift_139667+20X_offer_Pension_gift_139696+20X_offer_Pension_gift_139752+20X_offer_Pension_gift_139832+20X_offer_Pension_gift_139859+20X_offer_Pension_gift_139881+15X_offer_Pension_newsletter_139987+15X_offer_Pension_newsletter_140030+15X_offer_Pension_newsletter_140089+15X_offer_Pension_newsletter_140097+15X_offer_Pension_newsletter_139068+15X_offer_Pension_newsletter_139154+15X_offer_Pension_newsletter_139158+15X_offer_Pension_newsletter_139169+15X_offer_Pension_newsletter_139220+15

### total budget is equal to sume of products' budgets

In [106]:
mdl.add( budget_spent == mdl.sum(df1['budget_per_product']));

docplex.mp.LinearConstraint[](spent,EQ,X_budget_Savings+X_budget_Mortgage+X_budget_Pension)

### Balance the offers among products

In [112]:
for row in df1.itertuples():
    mdl.add_constraint(row.offering_budget <= row.budget_share *  total_offers)

### Do not exceed the budget

In [114]:
mdl.add_constraint( budget_spent  <= available_budget )  ;

# KPIs

We want to maximize the expected revenue.

In [135]:
df_ = pd.merge ( customer_offers, offers[['customerid','Product1','Confidence1']], left_on = ['customerid','product'], right_on =['customerid','Product1'] )
df_ = pd.merge(df_, channels, on = ['channel'])
df = pd.merge(df_, products, on = ['product'])

df.head()

,product,channel,customerid,channel_vars,Product1,Confidence1,cost,factor,value,budget_share,budget_per_product
0,Savings,gift,140030,X_offer_Savings_gift_140030,Savings,0.95678,20.0,0.2,200,0.2,X_budget_Savings
1,Savings,gift,140089,X_offer_Savings_gift_140089,Savings,0.95678,20.0,0.2,200,0.2,X_budget_Savings
2,Savings,gift,139532,X_offer_Savings_gift_139532,Savings,0.95676,20.0,0.2,200,0.2,X_budget_Savings
3,Savings,gift,139549,X_offer_Savings_gift_139549,Savings,0.16428,20.0,0.2,200,0.2,X_budget_Savings
4,Savings,gift,139560,X_offer_Savings_gift_139560,Savings,0.95678,20.0,0.2,200,0.2,X_budget_Savings
